In [1]:
pip install pyspark

  Using cached pyspark-3.2.1.tar.gz (281.4 MB)
  Using cached py4j-0.10.9.3-py2.py3-none-any.whl (198 kB)
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7fa5bb884bb2aa40cb6d4b36a7e8abc52f1366eeb2ad23f74e65faaa1c541265
  Stored in directory: c:\users\shiva\appdata\local\pip\cache\wheels\52\45\50\69db7b6e1da74a1b9fcc097827db9185cb8627117de852731e
Successfully built pyspark



In [1]:
import pyspark

In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('practice').getOrCreate()
spark

In [60]:
df = spark.read.csv('test.csv',header=True,inferSchema=True)
df,df.show()

+----+----+
|name| age|
+----+----+
|   a|  21|
|   b|  22|
|   c|  23|
|   d|  24|
|null|null|
|   f|null|
|   g|  25|
|   h|  26|
+----+----+



(DataFrame[name: string, age: int], None)

In [61]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [62]:
df.select(['name','age']).show()

+----+----+
|name| age|
+----+----+
|   a|  21|
|   b|  22|
|   c|  23|
|   d|  24|
|null|null|
|   f|null|
|   g|  25|
|   h|  26|
+----+----+



In [63]:
df.dtypes

[('name', 'string'), ('age', 'int')]

In [64]:
df.describe().show(),df.select('age').describe().show()

+-------+----+------------------+
|summary|name|               age|
+-------+----+------------------+
|  count|   7|                 6|
|   mean|null|              23.5|
| stddev|null|1.8708286933869707|
|    min|   a|                21|
|    max|   h|                26|
+-------+----+------------------+

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 6|
|   mean|              23.5|
| stddev|1.8708286933869707|
|    min|                21|
|    max|                26|
+-------+------------------+



(None, None)

In [65]:
df = df.withColumn('age after 5 years',df['age']+2)
df.show()

+----+----+-----------------+
|name| age|age after 5 years|
+----+----+-----------------+
|   a|  21|               23|
|   b|  22|               24|
|   c|  23|               25|
|   d|  24|               26|
|null|null|             null|
|   f|null|             null|
|   g|  25|               27|
|   h|  26|               28|
+----+----+-----------------+



In [66]:
df = df.drop('age after 5 years')
df.show()

+----+----+
|name| age|
+----+----+
|   a|  21|
|   b|  22|
|   c|  23|
|   d|  24|
|null|null|
|   f|null|
|   g|  25|
|   h|  26|
+----+----+



In [67]:
df = df.withColumnRenamed('age','age in years')
df.show()

+----+------------+
|name|age in years|
+----+------------+
|   a|          21|
|   b|          22|
|   c|          23|
|   d|          24|
|null|        null|
|   f|        null|
|   g|          25|
|   h|          26|
+----+------------+



In [68]:
df.na.drop().show(),df.na.drop(how= 'any').show(),df.na.drop('all').show()

+----+------------+
|name|age in years|
+----+------------+
|   a|          21|
|   b|          22|
|   c|          23|
|   d|          24|
|   g|          25|
|   h|          26|
+----+------------+

+----+------------+
|name|age in years|
+----+------------+
|   a|          21|
|   b|          22|
|   c|          23|
|   d|          24|
|   g|          25|
|   h|          26|
+----+------------+

+----+------------+
|name|age in years|
+----+------------+
|   a|          21|
|   b|          22|
|   c|          23|
|   d|          24|
|   f|        null|
|   g|          25|
|   h|          26|
+----+------------+



(None, None, None)

In [71]:
df = df.withColumn('age after 5 years',df['age in years']+2)
df.show()

+----+------------+-----------------+
|name|age in years|age after 5 years|
+----+------------+-----------------+
|   a|          21|               23|
|   b|          22|               24|
|   c|          23|               25|
|   d|          24|               26|
|null|        null|             null|
|   f|        null|             null|
|   g|          25|               27|
|   h|          26|               28|
+----+------------+-----------------+



In [78]:
#thresh means atleat N non null value should be there
N=1
df.na.drop(how= 'any',thresh = N).show()

+----+------------+-----------------+
|name|age in years|age after 5 years|
+----+------------+-----------------+
|   a|          21|               23|
|   b|          22|               24|
|   c|          23|               25|
|   d|          24|               26|
|   f|        null|             null|
|   g|          25|               27|
|   h|          26|               28|
+----+------------+-----------------+



In [85]:
df.na.drop(how= 'any',subset=['name']).show()

+----+------------+-----------------+
|name|age in years|age after 5 years|
+----+------------+-----------------+
|   a|          21|               23|
|   b|          22|               24|
|   c|          23|               25|
|   d|          24|               26|
|   f|        null|             null|
|   g|          25|               27|
|   h|          26|               28|
+----+------------+-----------------+



In [104]:
from pyspark.ml.feature import Imputer

imputer1 = Imputer(inputCols=['age in years','age after 5 years'],
                 outputCols=["{}_new".format(c) for c in ['age in years','age after 5 years']]
            ).setStrategy('mean')

imputer2 = Imputer(inputCols=['age in years','age after 5 years'],
                 outputCols=["{}_new".format(c) for c in ['age in years','age after 5 years']]
            ).setStrategy('median')

imputer3 = Imputer(inputCols=['age in years','age after 5 years'],
                 outputCols=["{}_new".format(c) for c in ['age in years','age after 5 years']]
            ).setStrategy('mode')

In [105]:
imputer1.fit(df).transform(df).show(),
imputer2.fit(df).transform(df).show(),
imputer3.fit(df).transform(df).show()

+----+------------+-----------------+----------------+---------------------+
|name|age in years|age after 5 years|age in years_new|age after 5 years_new|
+----+------------+-----------------+----------------+---------------------+
|   a|          21|               23|              21|                   23|
|   b|          22|               24|              22|                   24|
|   c|          23|               25|              23|                   25|
|   d|          24|               26|              24|                   26|
|null|        null|             null|              23|                   25|
|   f|        null|             null|              23|                   25|
|   g|          25|               27|              25|                   27|
|   h|          26|               28|              26|                   28|
+----+------------+-----------------+----------------+---------------------+

+----+------------+-----------------+----------------+---------------------

In [109]:
df = df.na.drop()
df.show()

+----+------------+-----------------+
|name|age in years|age after 5 years|
+----+------------+-----------------+
|   a|          21|               23|
|   b|          22|               24|
|   c|          23|               25|
|   d|          24|               26|
|   g|          25|               27|
|   h|          26|               28|
+----+------------+-----------------+



In [137]:
df.filter(df['age in years']<=23).select(['name','age after 5 years']).show(),
df.filter((df['age in years']<=23) & (df['age in years']>=21)).select(['name','age after 5 years']).show(),
df.filter(~(df['age in years']<=23)).select(['name','age after 5 years']).show(),
df.filter(df['age in years']<=23).collect()

+----+-----------------+
|name|age after 5 years|
+----+-----------------+
|   a|               23|
|   b|               24|
|   c|               25|
+----+-----------------+

+----+-----------------+
|name|age after 5 years|
+----+-----------------+
|   a|               23|
|   b|               24|
|   c|               25|
+----+-----------------+

+----+-----------------+
|name|age after 5 years|
+----+-----------------+
|   d|               26|
|   g|               27|
|   h|               28|
+----+-----------------+



[Row(name='a', age in years=21, age after 5 years=23),
 Row(name='b', age in years=22, age after 5 years=24),
 Row(name='c', age in years=23, age after 5 years=25)]